In [42]:
import sys

sys.path.insert(0, "../src")

import numpy
from typing import Union
from numpy import int32, int64, float32, float64, complex64, complex128
from abc import ABC
from dataclasses import dataclass
from enum import Enum
from functools import wraps
from timeserie.common import _typing


class Detectors(Enum):
    L1 = "Ligo Livingston (L1)"
    H1 = "Ligo Hanford (H1)"
    V1 = "Virgo (V1)"
    _ = "_"

@dataclass
class _BaseSeriesAttrs:
    segment_name: str
    detector: Detectors
    t0_gps: float64
    duration: float64
    dt: float64
    sampling_rate: int64

    def __post_init__(self):

        # Check for data type
        # ===================
        if not isinstance(self.segment_name, str):
            raise ValueError(f"segment_name must be a string.")
        if not isinstance(self.detector, Detectors):
            raise ValueError(f"detector must be a Detector.")
        if not isinstance(self.t0_gps, float64):
            raise ValueError(f"t0_gps must be a float64.")
        if not isinstance(self.duration, float64):
            raise ValueError(f"duration must be a float64.")
        if not isinstance(self.dt, float64):
            raise ValueError(f"dt must be a float64.")
        if not isinstance(self.sampling_rate, int64):
            raise ValueError(f"sampling_rate must be a int64.")

        # Checks for data integrity
        # =========================
        # sampling rate must power of 2, smaller than 4096
        _ = numpy.log2(self.sampling_rate) % 1 != 0 or self.sampling_rate > 4096
        if _:
            raise ValueError(f"sampling rate must be a power of 2, smaller than 4096")
        # dt and sampling rate compatibility
        _ = 1 / self.dt != self.sampling_rate
        if _:
            raise ValueError(f"dt and sampling_rate are not compatible.")
        # t0 compatibility
        _ = self.t0_gps != numpy.floor(self.t0_gps / self.dt) * self.dt
        if _:
            raise ValueError(f"t0_gps is incompatible with dt and sampling rate")
        # duration compatibility
        _ = self.duration != numpy.ceil(self.duration / self.dt) * self.dt
        if _:
            raise ValueError(f"duration is incompatible with dt and sampling rate")
    
    # modifying the setter to ensure read-only behaviour
    def __setattr__(self, __name: str, __value) -> None:
        if hasattr(self, __name):
            raise PermissionError(f"Attributes are readonly!")
        else:
            super().__setattr__(__name, __value)


@dataclass
class GPS_Interval:
    time_range: list[_typing.REAL_NUM, _typing.REAL_NUM]

    def __post_init__(self):
        # check input type
        if not isinstance(self.time_range, list):
            raise ValueError(f"Range must be a list of numbers.")
        # check input shape
        if len(self.time_range) != 2:
            raise ValueError(f"Range must be a list with 2 elements.")
        # check list content
        if not isinstance(self.time_range[0], _typing.REAL_NUM) or not isinstance(
            self.time_range[1], _typing.REAL_NUM
        ):
            raise ValueError(f"Range values must be real numbers.")
        # check time integrity
        if self.time_range[0] >= self.time_range[1]:
            raise ValueError(f"Start time must be smaller than stop time.")

In [63]:
from abc import ABC, abstractmethod, abstractproperty
class _BaseTimeSerie(ABC): 
    @abstractproperty
    def strain(self): pass

    @abstractproperty
    def times(self): pass

    @abstractproperty
    def attrs(self): pass

    @abstractproperty
    def nbytes(self): pass

    @abstractmethod
    def crop(self, gps_interval: GPS_Interval): pass

    @abstractmethod
    def resample(self, new_sampling_rate: _typing.INT_LIKE): pass

    @abstractmethod
    def whiten(self): pass
    
    @abstractmethod
    def fft(self): pass

    @abstractmethod
    def fftfreq(self): pass

    @abstractmethod
    def __repr__(self): pass

class CPUSerie(_BaseTimeSerie): ...


In [59]:
samprate = 1024
attr = _BaseSeriesAttrs(
    "_",
    Detectors.H1,
    float64(0),
    float64(1024),
    float64(1/samprate),
    int64(samprate),
)
attr.duration

1024.0

In [61]:
CPUSerie()

TypeError: _BaseSeriesAttrs.__init__() missing 6 required positional arguments: 'segment_name', 'detector', 't0_gps', 'duration', 'dt', and 'sampling_rate'

In [18]:
import numpy
arr = numpy.arange(0, 100, 1)
numpy.unique(numpy.diff(arr))[0]

1

In [6]:
a = None
b = 2.1

if not a and b:
    print("1")
elif b is None and a:
    print("2")
elif a and b:
    print("3")
else:
    print("4")

1
